<a href="https://colab.research.google.com/github/hymansun/slh_repo/blob/main/paraformer_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install pydub
!pip install fastapi
# !pip install modelscope
!pip install pyngrok
!pip install uvicorn
# !pip install funasr
!pip install git+https://github.com/openai/whisper.git

# !pip uninstall -y numpy
# !pip install numpy==1.22.0
# !pip uninstall -y pyarrow
# !pip install pyarrow==3.0.0

# https://3b2c-34-80-60-76.ngrok-free.app/asr?url=https://www.xiaoyuzhoufm.com/episode/644b8e9db7b3c21b651f4eff&act=TEXT
# https://22fd-35-240-130-155.ngrok-free.app/asr?url=https://www.xiaoyuzhoufm.com/episode/6480423c6752b5f9de289b49&act=TEXT

In [ ]:
!ngrok config add-authtoken 2S0vnfxJ0gGEOuFKszCI2GkITM8_3VugUQjVgDAsyvUiWcrEb

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# !pip uninstall -y numpy
# !pip install numpy==1.22.0
# !pip uninstall -y librosa
# !pip install librosa==0.8.1

In [ ]:
# !pip install git+https://github.com/m1guelpf/yt-whisper.git
# !pip install whisper
# !pip install git+https://github.com/openai/whisper.git

In [ ]:
!pip show whisper


In [ ]:
!pip show torch

In [ ]:
import os

import requests
from bs4 import BeautifulSoup
from pydub import AudioSegment
import requests
import re
import json
import requests
from fastapi import FastAPI
import whisper
import numpy
# from modelscope.pipelines import pipeline
# from modelscope.utils.constant import Tasks

import nest_asyncio
from pyngrok import ngrok
import uvicorn

save_data_service = 'http://139.198.168.88:8888/data'

# !pip install pydub
# !pip install fastapi
# !pip install modelscope
# !pip install pyngrok
# !pip install uvicorn
# https://79b4-35-233-188-114.ngrok-free.app/asr?url=https://www.xiaoyuzhoufm.com/episode/644b8e9db7b3c21b651f4eff&act=TEXT 谁提前给燃油车企出殡了？
# https://7c03-35-233-188-114.ngrok-free.app/asr?url=https://www.xiaoyuzhoufm.com/episode/64b0f90d44d572bfab0a74bb&act=TEXT 富士康败走印度，究竟“谁坑了谁”？
# https://ab96-103-116-72-184.ngrok-free.app/asr?url=https://www.xiaoyuzhoufm.com/episode/6480423c6752b5f9de289b49&act=TEXT
#  E13. 先实事求是，再去谈投资 | 对话南添
# !ngrok config add-authtoken 2S0vnfxJ0gGEOuFKszCI2GkITM8_3VugUQjVgDAsyvUiWcrEb


# =======================================

def get_audio_url(program_url):
    response = requests.get(program_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # Find the audio URL in the meta tags
    audio_url = None
    title = None
    description = None
    for meta in soup.find_all('meta'):
        if 'og:audio' in meta.attrs.get('property', ''):
            audio_url = meta.attrs.get('content', '')
            break
    return audio_url


def download_and_convert(audio_url):
    # 下载音频文件
    ori_file_name = re.search(r"/([^/]+\.\w+)$", audio_url).group(1)
    out_file_name = ori_file_name + '.wav'
    if os.path.exists(out_file_name):
        return out_file_name
    response = requests.get(audio_url)
    with open(ori_file_name, 'wb') as file:
        file.write(response.content)
    # 转换音频文件格式
    audio = AudioSegment.from_file(ori_file_name)
    audio.export(out_file_name, format='wav')
    return out_file_name

def get_wav_transcript_paraformer(wav_file):
    p = pipeline('auto-speech-recognition',
                 'damo/speech_paraformer-large-vad-punc_asr_nat-zh-cn-16k-common-vocab8404-pytorch')
    ts = p(wav_file)
    return ts


def get_wav_transcript_whisper(wav_file):
    options = {}
    options["language"] = "chinese"
    model = whisper.load_model('large-v2', device='cuda')
    result = model.transcribe(wav_file, verbose=False,
                              append_punctuations= "\"'.。,，!！?？:：”)]}、",
                              **options)
    print(result)
    return result


app = FastAPI()

@app.get("/asr")
def asr(url: str, act: str):
    if act.upper() == "TEXT":
        print(url)
        audio_url = get_audio_url(url)
        if audio_url == '':
            return "节目地址获取错误"
        out_file_name = download_and_convert(audio_url)
        if out_file_name == '':
            return "文件下载错误"
        ts =  get_wav_transcript_whisper(out_file_name)
        save_content(url, ts)
        s = ts['text']
        # print(ts)
        return s
    else:
        return {"url": url, "act": act, "result": "other content"}


def save_content(filename, ts):
    data = ts
    # filename = filename.replace("http://", "").replace("https://", "")
    params = {
        'filename': filename
    }
    response = requests.post(save_data_service, params=params, data=json.dumps(data))
    print(response)
    print(response.status_code)
    return response.status_code


ngrok_tunnel = ngrok.connect(8000)
print("Public URL:", ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)



Public URL: https://95e1-35-233-188-114.ngrok-free.app


INFO:     Started server process [724]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


https://www.xiaoyuzhoufm.com/episode/64b0f90d44d572bfab0a74bb
[00:00.000 --> 00:03.080] 富士康败走印度 究竟谁坑了谁
[00:03.080 --> 00:06.040] 本来来自虎岫科技组 作者王都山
[00:06.040 --> 00:08.760] 你好 我是本栏目主薄金涛
[00:08.760 --> 00:10.040] 2022年2月
[00:10.040 --> 00:11.880] 富士康母公司鸿海集团
[00:11.880 --> 00:13.840] 与印度矿业巨头维丹塔
[00:13.840 --> 00:16.720] 签署了共同建造芯片厂的协议
[00:16.720 --> 00:19.920] 那个被莫迪政府寄予厚望的印度造芯计划
[00:19.920 --> 00:21.320] 也被推向高潮
[00:21.320 --> 00:23.240] 但这份雄伟蓝图现在
[00:23.240 --> 00:25.520] 似乎只能草率收场了
[00:25.520 --> 00:27.120] 今年7月10日晚间
[00:27.120 --> 00:28.840] 鸿海集团发布公告称
[00:28.840 --> 00:30.960] 为了探索更多元的发展机会
[00:30.960 --> 00:32.640] 根据双方签订的协议
[00:32.640 --> 00:36.200] 鸿海后续将不再参与双方的合资公司运作
[00:36.200 --> 00:39.040] 这份公告的措辞给双方留足了体面
[00:39.040 --> 00:40.840] 但表现在实际行动上
[00:40.840 --> 00:43.520] 鸿海的这次退出没有一丝犹豫
[00:43.520 --> 00:45.560] 按照公告 双方的合资公司
[00:45.560 --> 00:47.880] 未来将完全由维丹塔集团持有
[00:47.880 --> 00:50.040] 并且鸿海已通知维丹塔
[00:50.040 --> 00:52.440] 移除合资公司中鸿海的名称
[00:52.440 --> 00:55.520] 以避免双方利益相关者混淆
[00:55.520 -->

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [724]
